In [438]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import pytz

In [23]:
data = [{'key_word':'明天', 
         'yy': -1, 'mm':-1,'dd':-1,'H':-1,'M':-1,'W':-1,'w':-1,
        'dyy':-1,'dmm':-1,'ddd':1,'dH':-1,'dM':-1,'dW':-1,},
        {'key_word':'下周一', 
         'yy': -1, 'mm':-1,'dd':-1,'H':-1,'M':-1,'W':-1,'w':1,
        'dyy':-1,'dmm':-1,'ddd':1,'dH':-1,'dM':-1,'dW':1,},]
# yy - year
# mm - month
# dd - day of month
# Hour
# M minute
# W week of a year
# day in a week

In [92]:
data = [{'key_word':'明天', 
         'expression':'?y-?m-+1d-12H:00M:00S'},
        {'key_word':'下周一', 
         'expression':'?y-+1W-1w-12H:00M:00S'},]

df = pd.DataFrame(data).set_index('key_word')
t1 = df.T.to_dict()['明天']
t2 = df.T.to_dict()['下周一']

In [ ]:
Process Sequence: S,M,H,D,Y,M / S,M,H,Y,w,W

In [552]:
utc = pytz.utc
tz = pytz.timezone("America/New_York")
# tz = pytz.timezone("Asia/Shanghai")
now = dt.datetime.now()
utc_now = utc.localize(now)
tz_now = tz.localize(now)
delta =  utc_now - tz_now 
hours = round(delta.total_seconds() / 3600)
delta = dt.timedelta(hours=hours)
tz_normal = dt.timezone(delta)

In [704]:
# yy - year
# mm - month
# dd - day of month
# Hour
# M minute
# W week of a year
# day in a week

# check week
current = dt.datetime.now()




current = tz.localize(current)
expression = '?y-+2W-3w-12H:00M:00S'
expression = a

exp_week = re.findall(r'-.+W-.+w',expression)
exp_ymd = re.findall(r'.+y-.+m.+d',expression)


history = pros_second(expression, current)
history = pros_minute(expression, current, history)
history,shift = pros_hour(expression, current, history)


if exp_week:
    #1. process week:
    
    history = pros_weekDay(expression,current, history)
    history = pros_week(expression,current, history)
    history = pros_year(expression, current,history)
    future = create_from_W(history)
    
elif exp_ymd:
    history = pros_day(expression, current,history)
    history = pros_month(expression, current,history)
    history = pros_year(expression, current,history)
    future = create_from_D(history)
if future.tzinfo is None:
    future = tz.localize(future)
if not shift:
    future = future - delta
print(future)


    
# processing year


2019-12-23 18:19:28-05:00


In [643]:
future.tzinfo

<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>

In [549]:
2018-06-25 19:51:55-04:00

<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>

In [561]:
dt.datetime.now()

datetime.datetime(2018, 6, 25, 15, 54, 41, 390163)

In [510]:
pytz.utc

<UTC>

In [519]:
utc = pytz.utc
now1 = dt.datetime.now()
now1 = utc.localize(now1)
now2 = dt.datetime.now()
now2 = tz.localize(now2)
b = now1 - now2

In [526]:

math.ceil(b.total_seconds()) / 3600

8.0

In [528]:
round(-3.9)

-4

## Process Week

In [635]:
expression='+1y-+30W-1w-12H:00M:00S'
def pros_week(expression, current, history):
    history = history.copy()
    year_adjust = 0
    W = current.isocalendar()[1] 
    reexp = r'y-.+W'
    extract = re.findall(reexp,expression)[0]
    
    # y-+1W
    if extract[2:-1] == '?':
        rep = str(W)
        history.update({'week':rep})
        return history
    elif extract[2] == '+':
        cur = W + int(extract[3:-1])
        if cur > 53:
            year_adjust = int(cur / 53)
            cur = cur % 53
        rep = str(cur)
        history.update({'year':year_adjust})
        history.update({'week':rep})
        return history
    elif extract[2] == '-':
        cur = W - int(extract[3:-1])
        if cur < 0:
            year_adjust = int(cur / 53) -1
            cur = cur % 53
        rep = str(cur)
        history.update({'year':year_adjust})
        history.update({'week':rep})
        return history
    else:
        rep = extract[2:-1]
        history.update({'week':rep})
        return history
        


pros_week(expression, current,history)

{'day': 25,
 'hour': 21,
 'microsecond': 0,
 'minute': 34,
 'month': 6,
 'second': 7,
 'week': '3',
 'year': 1}

In [474]:
dt.timezone.utc

datetime.timezone.utc

## Process Week Day

In [626]:
expression='?y-+2W-2w-12H:00M:00S'
def pros_weekDay(expression, current, history):
    history = history.copy()
    w = current.isocalendar()[2] 
    reexp = r'W-.+w'
    extract = re.findall(reexp,expression)[0]
    
    # W-+1w
    if extract[2:-1] == '?':
        history.update({'weekday':str(w)})
        return history
    elif extract[2] == '+':
        rep = str(w + int(extract[3:-1]))
        history.update({'weekday':rep})
        return history
    elif extract[2] == '-':
        rep = str(w - int(extract[3:-1]))
        history.update({'weekday':rep})
        return history
    else:
        rep = extract[2:-1]
        history.update({'weekday':rep})
        return history
pros_weekDay(expression, current,history)

{'day': 25,
 'hour': 21,
 'microsecond': 0,
 'minute': 34,
 'month': 6,
 'second': 7,
 'weekday': '3',
 'year': 2018}

## Process Year

In [427]:
expression = '-5y-?m-+1d-12H:00M:?S'
def pros_year(expression, current, history):
    if isinstance(history,dt.datetime):
        return history
    history = history.copy()
    adjust_year = history.get('year')
    if adjust_year is None:
        adjust_year = 0
    y = current.year
    reexp = r'.+y-'
    extract = re.findall(reexp,expression)[0]
    if extract[0:-2] == '?':
        history.update({'year':y+adjust_year})
        return history
    elif extract[0] == '+':
        gap = int(extract[1:-2])
        history.update({'year':y+adjust_year+gap})
        return history
    elif extract[0] == '-':
        gap = int(extract[1:-2])
        history.update({'year':y+adjust_year-gap})
        return history
    else:
        history.update({'year':int(extract[:-2])+adjust_year})
        return history
pros_year(expression, current, history)

datetime.datetime(2013, 12, 26, 12, 0)

In [412]:
history

{'hour': 2,
 'microsecond': 0,
 'minute': 12,
 'month': 12,
 'second': 22,
 'year': 2013}

## Process Month

In [447]:
expression = '-5y-+1m-+1d-12H:00M:?S'
def pros_month(expression, current, history):
    if isinstance(history,dt.datetime):
        return history
    history = history.copy()
    adjust_year = 0
    m = current.month
    reexp = r'y-.+m'
    extract = re.findall(reexp,expression)[0]
    if extract[2:-1] == '?':
        history.update({'month':m})
        return history
    elif extract[2] == '+':
        cur = int(extract[3:-1]) + m
        if cur > 12:
            adjust_year = int(cur / 12)
            cur = cur % 12
            if cur == 0:
                cur = 12
                adjust_year -= 1
        history.update({'year':adjust_year})
        history.update({'month':cur})
        return history
    elif extract[2] == '-':
        cur = m - int(extract[3:-1])
        if cur < 1:
            adjust_year = int(cur / 12) - 1
            cur = cur % 12
            if cur == 0:
                cur = 12
        history.update({'year':adjust_year})
        history.update({'month':cur})
        return history
    else:
        history.update({'month':int(extract[2:-1])})
        return history
pros_month(expression, current, history)

{'day': 11,
 'hour': 11,
 'microsecond': 0,
 'minute': 47,
 'month': 7,
 'second': 14,
 'year': 0}

In [387]:
history

{'hour': 2,
 'microsecond': 0,
 'minute': 12,
 'month': 12,
 'second': 22,
 'year': 2013}

## Processing second

In [590]:
re.findall(r'M:.+S','?y-?m-?d-?H:31M:33S')

['M:33S']

In [591]:
expression = '?y-?m-?d-?H:31M:33S'
#1. process seconds

def pros_second(expression, current, history={'microsecond':0}):
    history = history.copy()
    S = current.second
    reexp = r'M:.+S'
    extract = re.findall(reexp,expression)[0]
    # M:+1S
    if extract[2:-1] == '?':
        history.update({'second':S})
        return history
    elif extract[2] == '+':
        gap = int(extract[3:-1])
        create = current + dt.timedelta(seconds=gap) 
        create = create.replace(**history)
        return create 
    elif extract[2] == '-':
        gap = int(extract[3:-1])
        create =  current - dt.timedelta(seconds=gap)
        create = create.replace(**history)
        return create
    else:
        second = int(extract[2:-1])
        history.update({'second':second})
        return history
history = pros_second(expression, current)


## Processing Minutes

In [577]:
expression = '?y-?m-+1d-12H:12M:00S'
#1. process seconds

def pros_minute(expression, current, history = {}):
    if isinstance(history,dt.datetime):
        return history
    history = history.copy()
    M = current.minute
    reexp = r'H:.+M'
    extract = re.findall(reexp,expression)[0]
    # H:?M
    if extract[2:-1] == '?':
        history.update({'minute':M})
        return history
    elif extract[2] == '+':
        gap = int(extract[3:-1])
        create = current + dt.timedelta(minutes=gap) 
        create = create.replace(**history)
        return create
    elif extract[2] == '-':
        gap = int(extract[3:-1])
        create = current - dt.timedelta(minutes=gap) 
        create = create.replace(**history)
        return create
    else:
        minute = int(extract[2:-1])
        history.update({'minute':minute})
        return history
history = pros_minute(expression, current, history)
history

datetime.datetime(2018, 6, 25, 16, 15, 56, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)

## Processing Hours

In [568]:
expression = '?y-?m-+1d-2H:+20M:00S'
#1. process seconds

def pros_hour(expression, current, history = {}):
    shift = True
    if isinstance(history,dt.datetime):
        return history, shift
    history = history.copy()
    
    H = current.hour
    reexp = r'[dw]-.+H'
    extract = re.findall(reexp,expression)[0]
#     d-?H
    if extract[2:-1] == '?':
        history.update({'hour':H})
        return history,shift
    elif extract[2] == '+':
        gap = int(extract[3:-1])
        create = current + dt.timedelta(hours=gap)
        create = create.replace(**history)
        return create,shift
    elif extract[2] == '-':
        gap = int(extract[3:-1])
        create = current - dt.timedelta(hours=gap)
        create = create.replace(**history)
        return create,shift
    else:
        shift = False
        hour = int(extract[2:-1])
        history.update({'hour':hour})
        return history,shift
history,shift = pros_hour(expression, current, history)
history

AttributeError: 'tuple' object has no attribute 'copy'

## Processing Days

In [422]:
expression = '?y-?m--1d-12H:+20M:00S'
#1. process seconds

def pros_day(expression, current, history={}):
    if isinstance(history,dt.datetime):
        return history
    history = history.copy()
    d = current.day
    reexp = r'm-.+d'
    extract = re.findall(reexp,expression)[0]
    if extract[2:-1] == '?':
        history.update({'day':d})
        return history
    elif extract[2] == '+':
        gap = int(extract[3:-1])
        create = current + dt.timedelta(days=gap) 
        create = create.replace(**history)
        return create
    elif extract[2] == '-':
        gap = int(extract[3:-1])
        create = current - dt.timedelta(days=gap) 
        create = create.replace(**history)
        return create
    else:
        day = int(extract[2:-1])
        history.update({'day':day})
        return history
pros_day(expression, current)

datetime.datetime(2018, 6, 24, 14, 32, 32, 123075)

In [416]:
isinstance(a,dt.datetime)

True

In [186]:
re.findall(r'[dw]-.+H','?y-?m-+1d-12H:00M:00S')

['d-12H']

# Create Time from Week

In [636]:
def create_from_W(history):
    expression ='{}y-{}W-{}w-{}H:{}M:{}S'.format(history['year'],
                                                 history['week'],
                                                 history['weekday'],
                                                 history['hour'],
                                                 history['minute'],
                                                 history['second'])
    eval_time = dt.datetime.strptime(expression, "%Yy-%WW-%ww-%HH:%MM:%SS")
    return eval_time

In [449]:
def create_from_D(history):
    if isinstance(history,dt.datetime):
        return history
    return dt.datetime(**history)
create_from_D(history)

datetime.datetime(2018, 6, 11, 11, 47, 38)

## Update time by history

In [362]:
a = dt.datetime.now()+dt.timedelta(month=1)
history.update({'microsecond':0})
a.replace(**history)

TypeError: 'month' is an invalid keyword argument for this function

In [155]:
a = dt.datetime.now()+dt.timedelta(days=1)
a

datetime.datetime(2018, 6, 26, 1, 46, 48, 862445)

In [157]:
b = dt.datetime.now()
a = dt.datetime.now()+dt.timedelta(seconds=1)
print(b)
print(a)

2018-06-25 01:47:34.171666
2018-06-25 01:47:35.171711


In [247]:
a.replace(year=2019,month=5,day=21,hour=12,minute=0,second=0,microsecond=0)

datetime.datetime(2019, 5, 21, 12, 0)

In [124]:
dt.datetime.now()

datetime.datetime(2018, 6, 24, 3, 24, 15, 751450)

In [125]:
b = create_from_D('2018y-6m-14d-12H:00M:00S')

In [126]:
b.microsecond

0

In [132]:
current.day

24

In [409]:
0+None

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [708]:
a = '我下个月23日还'
re.findall(r'(?:明年|今年|后年)?(?:\d{1,2}|十二|十一|十|九|八|七|六|五|四|三|二|一|下个|再下个)*月\d{1,2}[日号]',a)

['下个月23日']

In [703]:
a = '{}y-{}m-{}d-{}H:{}M:{}S'.format('+1',12,23,'?','?','?')
a

'+1y-12m-23d-?H:?M:?S'

# DEV BY SHU

In [764]:
import re

def reg(string):
    test = re.compile(r'(?:(?:今|明|后|大后)年)?(?:(?:\d{1,2}|下个|下下个|下下下个|一|二|三|四|五|六|七|八|九|十|十一|十二)月)?(?:(?:\d{1,2}|一|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十|三十一)日)?')
    result = re.findall(test,string)[0]
    return result
   # sampl = '{}y-{}m-{}d-{}H:{}M:{}S'.format()
def to_form(input):
    if input is None:
        input = '?'
    return input
def split(result):
    time_dic = {'今':'0','明':'+1','后':'+2','大后':'+3','下个':'+1','下下个':'+2','下下下个':'+3','一':'1','二':'2','三':'3','四':'4','五':'5','六':'6',
               '七':'7','八':'8','九':'9','十':'10','十一':'11','十二':'12','十三':'13','十四':'14','十五':'15',
               '十六':'16','十七':'17','十八':'18','十九':'19','二十':'20','二十一':'21','二十二':'22','二十三':'23',
               '二十四':'24','二十五':'25','二十六':'26','二十七':'27','二十八':'28','二十九':'29','三十':'30','三十一':'31'}

    year_index = result.find('年')
    month_index = result.find('月')
    if result.find('日') != -1:
        date_index = result.find('日')
    else:
        date_index = result.find('号')

    if year_index != -1:
        year_key = result[0:year_index]  
        year = time_dic.get(year_key)
        year = to_form(year)

    else:
        year = '?'
    if month_index != -1:
        month_key = result[year_index+1:month_index]
        month = time_dic.get(month_key)
        month = to_form(month)
    else:
        month = '?'
    if date_index != -1:
        date_key = result[month_index+1:date_index]
        date = time_dic.get(date_key)
        date = to_form(date)
    else:
        date = '?'
    print(year,month,date)


split(reg('下下个月二十四日还'))

# re.findall(r'(?:(明|今|后)年)?(?:\d{1,2}|十二|十一|十|九|八|七|六|五|四|三|二|一)*(?:月)*\d{1,2}[日号]',a)

? +2 24


In [802]:
def reg(x):
    fix_ymd = r'(?:(?:今|明|后|大后)年)?(?:(?:\d{1,2}|下下下个|下下个|再下个|下个|十一|十二|一|二|三|四|五|六|七|八|九|十|后1个|后2个|后一个|后两个|后二个)月)?(?:(?:\d{1,2}|一|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十|三十一)[日号])?'
    finds = list(set(re.findall(fix_ymd,x)) -set(['']))
    return finds
    

reg('我下个月6号就还，哦不不下下个月4号')

['下下个月4号', '下个月6号']

In [846]:


def reg(x):
    fix_ymd = r'(?:(?:今|明|后|大后)年)?(?:(?:\d{1,2}|下下下个|下下个|再下个|下个|十一|十二|一|二|三|四|五|六|七|八|九|十|后1个|后2个|后一个|后两个|后二个)月)?(?:(?:\d{1,2}|一|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十|三十一)[日号])?'
    finds = list(set(re.findall(fix_ymd,x)) -set(['']))
    return finds



def ymd_expression(result):
    def get_key(x):
        if x.isdigit():
            return x
        else:
            gets = time_dic.get(x)
            if x is None:
                return '?'
            else:
                return gets
            
    
    time_dic = {'今':'0','明':'+1','后':'+2','大后':'+3','下个':'+1','下下个':'+2','再下个':'+2','下下下个':'+3','后1个':'+1','后2个':'+1','后一个':'+1','后两个':'+1',
                '一':'1','二':'2','三':'3','四':'4','五':'5','六':'6',
               '七':'7','八':'8','九':'9','十':'10','十一':'11','十二':'12','十三':'13','十四':'14','十五':'15',
               '十六':'16','十七':'17','十八':'18','十九':'19','二十':'20','二十一':'21','二十二':'22','二十三':'23',
               '二十四':'24','二十五':'25','二十六':'26','二十七':'27','二十八':'28','二十九':'29','三十':'30','三十一':'31'}
    year_index = result.find('年')
    month_index = result.find('月')
    if result.find('日') != -1:
        date_index = result.find('日')
    else:
        date_index = result.find('号')
        
    if year_index != -1:
        year_key = result[0:year_index]  
        year = get_key(year_key)
    else:
        year = '?'
    if month_index != -1:
        month_key = result[year_index+1:month_index]
        month = get_key(month_key)
    else:
        month = '?'
    if date_index != -1:
        date_key = result[month_index+1:date_index]
        date = get_key(date_key)
    else:
        date = '?'
    formatted = '{}y-{}m-{}d-12H:00M:00S'.format(year, month, date)
    return formatted


def evl_ymd(text):
    finds = reg(text)
    evls = []
    if len(finds) == 0:
        return elvs
    else:
        for each in finds:
            evls.append({'pattern':each, 'expression':ymd_expression(each)})
    return evls

In [848]:
evl_ymd('下个月二十三号可能不行，明年7月份还')


[{'expression': '+1y-7m-?d-12H:00M:00S', 'pattern': '明年7月'},
 {'expression': '?y-+1m-23d-12H:00M:00S', 'pattern': '下个月二十三号'}]

In [809]:
to_form = lambda x: '?' if x is None else x

In [814]:
to_form('123')

'123'

In [829]:
a = '01'

In [830]:
a.isdigit()

True